# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *K*

**Names:**

* *Robin Lang*
* *Kim Lan Phan Hoang*
* *Julien Harbulot*

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import epidemics_helper
import json

import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import networkx as nx
from networkx.readwrite import json_graph

---

## 2.4 PageRank

### 2.4.1 Random Surfer Model

#### Exercise 2.12

In [2]:
# open the files and store them as graphs
absorbing_file = open("../data/absorbing.graph", "rb")
absorbing_graph = nx.read_adjlist(absorbing_file, create_using=nx.DiGraph())

components_file = open("../data/components.graph", "rb")
components_graph = nx.read_adjlist(components_file, create_using=nx.DiGraph())

wikipedia_file = open("../data/wikipedia.graph", "rb")
wikipedia_graph = nx.read_adjlist(wikipedia_file, create_using=nx.DiGraph())

G_n1 = nx.read_edgelist('../data/network1.csv', delimiter=',', nodetype=int, encoding="utf-8")

In [4]:
def random_surfer(G, N = 5):
    nodes = nx.nodes(G)
    num_nodes = len(nodes)
    prob = np.zeros(num_nodes)
    first_node = random.choice(nodes)
    
    succ = G.successors(first_node)
    
    # have no neighbors
    if len(succ) == 0:
        prob[int(first_node)] = 1
        return prob
    
    # construct probability vector
    for n in succ:
        prob[int(n)] = 1/len(succ)
    
    # update vector when reaching a new node
    for i in range(N-1):
        prob_next = np.zeros(num_nodes)
        for j in nodes:
            if prob[int(j)] > 0:
                succ = G.successors(j)
                if len(succ) > 0:
                    for n in succ:
                        prob_next[int(n)] += prob[int(j)] / len(succ)
                else:
                    prob_next[int(j)] += prob[int(j)]
        prob = prob_next
    return prob

In [51]:
random_surfer(absorbing_graph)

array([ 0.03703704,  0.92592593,  0.03703704,  0.        ,  0.        ])

We obtain 3 kinds of matrix : [ 0.,  1.,  0.,  0.,  0.], 
[ 0.03703704,  0.92592593,  0.03703704,  0.        ,  0.        ], 
[ 0.        ,  0.83333333,  0.        ,  0.11111111,  0.05555556]

It shows that the node 1 has the higher score. It is due to the fact that all paths can finish at this node. Also, node 1 does not have a link to any node.

In [55]:
random_surfer(components_graph)

array([ 0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ])

#### Exercise 2.13

In [6]:
def method1(G, N = 5):
    nodes = nx.nodes(G)
    num_nodes = len(nodes)
    prob = np.zeros(num_nodes)
    first_node = random.choice(nodes)
    
    succ = G.successors(first_node)
    
    if len(succ) == 0:
        prob[int(first_node)] = 1
        return prob
    
    for n in succ:
        prob[int(n)] = 1/len(succ)
    
    for i in range(N-1):
        prob_next = np.zeros(num_nodes)
        for j in nodes:
            if prob[int(j)] > 0:
                succ = G.successors(j)
                if len(succ) > 0:
                    for n in succ:
                        prob_next[int(n)] += prob[int(j)] / len(succ)
                else:
                    prob_next[int(random.choice(nodes))] += prob[int(j)]
        prob = prob_next
    return prob

In [7]:
method1(absorbing_graph, 100)

array([ 0.06360044,  0.46061609,  0.06360044,  0.0096789 ,  0.40250413])

In [8]:
method1(components_graph, 100)

array([ 0.28571351,  0.28571278,  0.28571572,  0.14285799,  0.        ,
        0.        ,  0.        ,  0.        ])

In [9]:
def method2(G, N = 5, damp = 0.15):
    nodes = nx.nodes(G)
    num_nodes = len(nodes)
    prob = np.zeros(num_nodes)
    first_node = random.choice(nodes)
    
    succ = G.successors(first_node)
    
    if len(succ) == 0:
        prob[int(first_node)] = 1
        return prob
    
    for n in succ:
        prob[int(n)] = 1/len(succ)
        
    for i in range(N-1):
        if np.random.choice([True, False], p=[damp, 1-damp]):
            print("1-2-SWITCH")
        prob_next = np.zeros(num_nodes)
        for j in nodes:
            if prob[int(j)] > 0:
                succ = G.successors(j)
                if len(succ) > 0:
                    for n in succ:
                        prob_next[int(n)] += prob[int(j)] / len(succ)
                else:
                    prob_next[int(j)] += prob[int(j)]
        prob = prob_next
    return prob

In [10]:
method2(absorbing_graph, 100)

array([ 0.,  1.,  0.,  0.,  0.])

---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [3]:
# create the w vector, which initialize the position of dangling node to 1
def danglingNodes(G):
    nbNodes = nx.number_of_nodes(G)
    w = np.zeros([nbNodes,nbNodes]);
    
    for i in G.nodes_iter():
        outDegreeCurrent = G.out_degree(i)
        if(outDegreeCurrent == 0):
            for j in G.nodes_iter():
                w[int(i),int(j)] = 1
    return w


# create markov chain matrix, where position ij is the probability to go from i to j
def markovMatrix(G):
    nbNodes = G.number_of_nodes()
    H = np.zeros([nbNodes,nbNodes])
    
    for n in G.nodes_iter():
        outDegreeCurrent = G.out_degree(n)
        for i in G.neighbors(n):
            H[int(n),int(i)] = 1/outDegreeCurrent
            
    return H
    

# main power iteration
def powerIteration(G, theta):
    
    
    N = G.number_of_nodes()
    
    w = danglingNodes(G)
    H = markovMatrix(G)
    
    # initial vector pi_0
    matrix = np.empty(N)
    matrix.fill(1/N)
    
    # apply formula 
    H_hat = np.add(H, w/N)
    Google = (theta*H_hat) + ((1-theta)/N)
    
    count = 0
    while(count != 30):
        
        newMatrix = matrix @ Google
        
        # check for convergence
        if(np.array_equal(matrix, newMatrix)):
            count += 1
        else :
            count = 0
            
        matrix = newMatrix
    
    return matrix
            

In [4]:
power_it = powerIteration(wikipedia_graph,0.8)
#obtain the 10 best ids
best10 = power_it.argsort()[:10]
print(best10)

not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
[2214 1651 1661 1665 1673 1675 1676 1684 1692 1693]


Here are the corresponding book titles :

2214 - Godspell<br>
1651 - Dr. Seuss<br>
1661 - Dundee United F.C.<br>
1665 - Dunstable and Whipsnade Downs<br>
1673 - Dysphania ambrosioides<br>
1675 - €2 commemorative coins<br>
1676 - e (mathematical constant)<br>
1684 - Earwax<br>
1692 - Eastern Roman Empire<br>
1693 - East Flemish

---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*

In [5]:
print("Score of page History of matemathics:",power_it[2463])
print(np.shape(power_it.argsort()))

Score of page History of matemathics: 9.97890687703e-05
(5540,)


In [11]:
wikipedia_file1 = open("../data/wikipedia.graph", "rb")
G = nx.read_adjlist(wikipedia_file1, create_using=nx.DiGraph())

nbEdges = 30

for i in range(len(power_it.argsort()[:nbEdges])):
    toNode = power_it[i]
    G.add_edge(toNode,'2463')
    print(nx.number_of_edges(G))

powerIteration(G, 0.8)

197057
197058
197059
197060
197061
197062
197063
197064
197065
197066
197067
197068
197069
197070
197071
197072
197073
197074
197075
197076
197077
197078
197079
197080
197081
197082
197083
197084
197085
197086
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
no

KeyboardInterrupt: 

197087